# SETUP

## is GPU on?

In [1]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

## boilerplate

In [1]:
from tensorflow.keras.callbacks import Callback, EarlyStopping
from numpy import arange

MODEL_NAME = 'inception3'
IMG_SIZE = (299, 299)
INPUT_SHAPE=(299, 299, 3)
CLASSES = 2
TRIAL = list(arange(3,10)) #list(arange(0,3))
FT_BLOCK = list(arange(0,11)) # FROM feature extractor TO fine tuning scratch
BATCH_SIZE = [32, 64, 128, 256] # these data points will be passed as a batch at one time to the network

## UDC/Fs

In [2]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from math import floor
from timeit import default_timer as timer
from json import dump
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import data

def model_constructor(FT_BLOCK):
    base_model = InceptionV3(
        weights='imagenet', 
        include_top=False,
        input_shape=INPUT_SHAPE)
    
    # construct the top layer containing 
    x = base_model.output
    x = layers.GlobalAveragePooling2D(name='avg_pool')(x) # add a global spatial average pooling layer
    x = layers.Dense(CLASSES, activation='sigmoid',  name='predictions')(x) # and add the output layer for binary class

    # model is ready to be trained
    model = Model(inputs=base_model.input, outputs=x)

    # freeze the layers before the `FROZEN_END` layer 
    total_layers = len(base_model.layers)
    ft_layers = floor(total_layers / 10)
    FROZEN_END = total_layers - ft_layers * FT_BLOCK

    base_model.trainable = True
    for layer in base_model.layers[:FROZEN_END]: 
        layer.trainable = False
    
    return model

class TimeCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

def time_converter(sec):
    hours, rem = divmod(sec, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

def save_history(history, tag):
    file_path = f'{HISTORY_DIR}/{tag}.json'
    with open(file_path, 'w') as f:
        dump(history.history, f)
        
def data_preparation(BATCH_SIZE):
    train_dir = '../../data/binaryclass_clean/train/'
    test_dir = '../../data/binaryclass_clean/test/'

    train_ds = image_dataset_from_directory(
        directory=train_dir,
        label_mode='categorical',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        seed=0,
        validation_split=0.1,
        subset='training')

    val_ds = image_dataset_from_directory(
        directory=train_dir,
        label_mode='categorical',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        seed=0,
        validation_split=0.1,
        subset='validation')

    test_ds = image_dataset_from_directory(
        directory=test_dir,
        label_mode='categorical',
        batch_size=1,
        image_size=IMG_SIZE)

    AUTOTUNE = data.AUTOTUNE
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds, test_ds

# GRID SEARCH

In [4]:
for i in TRIAL:
    print(" - - - - - TRIAL:", i, " - - - - - ")
    HISTORY_DIR = f'../../logs/FT/{i}'
    for batch in BATCH_SIZE:
        print("Batch size:", batch)
        train_ds, val_ds, test_ds = data_preparation(batch)
        epochs, loss, accuracy = [None]*len(FT_BLOCK), [None]*len(FT_BLOCK), [None]*len(FT_BLOCK)
        for ft in FT_BLOCK:
            tag = f'{MODEL_NAME}_BS{batch}_FT{ft*10}'
            print(tag)
            model = model_constructor(ft)
            model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='adam')
            history = model.fit(train_ds, epochs=100, verbose=0, validation_data=val_ds, callbacks=[EarlyStopping(patience=3)])
            save_history(history, tag)
            # time_converter(sum(cb_time.logs))
            epochs[ft] = len(history.history['loss'])
            loss[ft], accuracy[ft] = model.evaluate(test_ds)
        # PRINT THE RESULT
        # print('Count Epoch:', epochs)
        print(f' * * * * * {MODEL_NAME}_BS{batch}_LOSS * * * * * ')
        for item in loss:
            print(item)
        print(f' * * * * * {MODEL_NAME}_BS{batch}_ACCURACY * * * * * ')
        for item in accuracy:
            print(item)

 - - - - - TRIAL: 0  - - - - - 
Batch size: 32
Found 9980 files belonging to 2 classes.
Using 8982 files for training.
Found 9980 files belonging to 2 classes.
Using 998 files for validation.
Found 1000 files belonging to 2 classes.
inception3_BS32_FT0
1000/1000 [==============================] - 17s 16ms/step - loss: 0.9100 - accuracy: 0.6030
inception3_BS32_FT10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.5750 - accuracy: 0.7240
inception3_BS32_FT20
1000/1000 [==============================] - 16s 16ms/step - loss: 0.5896 - accuracy: 0.6890
inception3_BS32_FT30
1000/1000 [==============================] - 28s 28ms/step - loss: 0.6314 - accuracy: 0.6660
inception3_BS32_FT40
1000/1000 [==============================] - 38s 38ms/step - loss: 0.5857 - accuracy: 0.6850
inception3_BS32_FT50
1000/1000 [==============================] - 38s 38ms/step - loss: 0.4174 - accuracy: 0.8380
inception3_BS32_FT60
1000/1000 [==============================] - 38s 37ms/step - lo

1000/1000 [==============================] - 21s 21ms/step - loss: 0.6513 - accuracy: 0.7060
inception3_BS32_FT30
1000/1000 [==============================] - 27s 27ms/step - loss: 0.6798 - accuracy: 0.6160
inception3_BS32_FT40
1000/1000 [==============================] - 60s 59ms/step - loss: 0.5109 - accuracy: 0.7420
inception3_BS32_FT50
1000/1000 [==============================] - 24s 24ms/step - loss: 0.5621 - accuracy: 0.6980
inception3_BS32_FT60
1000/1000 [==============================] - 26s 26ms/step - loss: 0.5974 - accuracy: 0.7590
inception3_BS32_FT70
1000/1000 [==============================] - 28s 28ms/step - loss: 0.4404 - accuracy: 0.8140
inception3_BS32_FT80
1000/1000 [==============================] - 27s 27ms/step - loss: 1.4722 - accuracy: 0.7290
inception3_BS32_FT90
1000/1000 [==============================] - 27s 27ms/step - loss: 0.6683 - accuracy: 0.7750
inception3_BS32_FT100
1000/1000 [==============================] - 26s 26ms/step - loss: 1.3014 - accuracy: 0